# TraitGym [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/songlab-cal/TraitGym/blob/main/TraitGym.ipynb)
In this example we will load the Mendelian traits dataset and run variant effect prediction based on euclidean distance of GPN-Animal-Promoter embeddings of the reference and alternate sequences.

## Setup

In [1]:
!pip install -q pyfaidx s3fs git+https://github.com/songlab-cal/gpn.git
!pip install -q -U transformers datasets

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.8/185.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from Bio.Seq import Seq
from datasets import load_dataset
import fsspec
import gpn.model  # to register AutoModel
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
from pyfaidx import Fasta
import seaborn as sns
from sklearn.metrics import average_precision_score
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer
import tempfile

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
dataset_path = "songlab/TraitGym"
dataset_config = "mendelian_traits"
model_path = "songlab/gpn-animal-promoter"
# in the paper we average with the predictions with the reverse complement
# however for a quick evaluation you can omit this
average_rc = False

## Load dataset

In [ ]:
dataset = load_dataset(dataset_path, dataset_config, split="test")
dataset

In [ ]:
dataset[0]

In [ ]:
# if you just want a dataframe you can load it directly:
# pd.read_parquet("hf://datasets/songlab/TraitGym/mendelian_traits_matched_9/test.parquet")
V = dataset.to_pandas()
V

## Load model

In [ ]:
# Wrapper to compute a VEP score
class ModelVEP(torch.nn.Module):
    def __init__(self, model_path):
        super().__init__()
        self.model = AutoModel.from_pretrained(
            model_path,
            trust_remote_code=True,
        )

    def forward(
        self,
        input_ids_ref=None,
        input_ids_alt=None,
    ):
        embed_ref = self.model(input_ids=input_ids_ref).last_hidden_state.reshape(len(input_ids_ref), -1)
        embed_alt = self.model(input_ids=input_ids_alt).last_hidden_state.reshape(len(input_ids_ref), -1)
        return F.pairwise_distance(embed_ref, embed_alt)


class ModelVEPAverageRC(torch.nn.Module):
    def __init__(self, model_path):
        super().__init__()
        self.model = AutoModel.from_pretrained(
            model_path,
            trust_remote_code=True,
        )

    def get_scores(self, input_ids_ref, input_ids_alt):
        embed_ref = self.model(input_ids=input_ids_ref).last_hidden_state.reshape(len(input_ids_ref), -1)
        embed_alt = self.model(input_ids=input_ids_alt).last_hidden_state.reshape(len(input_ids_ref), -1)
        return F.pairwise_distance(embed_ref, embed_alt)

    def forward(
        self,
        input_ids_ref_fwd=None,
        input_ids_alt_fwd=None,
        input_ids_ref_rev=None,
        input_ids_alt_rev=None,
    ):
        fwd = self.get_scores(input_ids_ref_fwd, input_ids_alt_fwd)
        rev = self.get_scores(input_ids_ref_rev, input_ids_alt_rev)
        return (fwd + rev) / 2

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
window_size = 512
if average_rc:
    model = ModelVEPAverageRC(model_path)
else:
    model = ModelVEP(model_path)

## Tokenize dataset

In [ ]:
class Genome:
    def __init__(self, path):
        self.data = Fasta(fsspec.open(path, anon=True))

    def __call__(self, chrom, start, end, strand="+"):
        res = self.data[chrom][start:end]
        if strand == "-":
            res = res.reverse.complement
        return str(res)

In [ ]:
genome = Genome("s3://broad-references/hg38/v0/Homo_sapiens_assembly38.fasta")

In [ ]:
def tokenize(seqs):
    return tokenizer(
        seqs,
        padding=False,
        truncation=False,
        return_token_type_ids=False,
        return_attention_mask=False,
        return_special_tokens_mask=False,
        return_tensors="pt",
    )["input_ids"]

def get_tokenized_seq(vs):
    # we convert from 1-based coordinate (standard in VCF) to
    # 0-based, to use with Genome
    chrom = np.array(vs["chrom"])
    n = len(chrom)
    pos = np.array(vs["pos"]) - 1
    start = pos - window_size // 2
    end = pos + window_size // 2
    seq_fwd = [genome("chr" + chrom[i], start[i], end[i]) for i in range(n)]
    seq_fwd = np.array([list(seq.upper()) for seq in seq_fwd], dtype="object")
    assert seq_fwd.shape[1] == window_size
    ref_fwd = np.array(vs["ref"])
    alt_fwd = np.array(vs["alt"])
    pos_fwd = window_size // 2

    def prepare_output(seq, pos, ref, alt):
        assert (seq[:, pos] == ref).all(), f"{seq[:, pos]}, {ref}"
        seq_ref = seq
        seq_alt = seq.copy()
        seq_alt[:, pos] = alt
        return (
            tokenize(["".join(x) for x in seq_ref]),
            tokenize(["".join(x) for x in seq_alt]),
        )

    res = {}
    res["input_ids_ref"], res["input_ids_alt"] = prepare_output(seq_fwd, pos_fwd, ref_fwd, alt_fwd)
    return res

def get_tokenized_seq_average_rc(vs):
    # we convert from 1-based coordinate (standard in VCF) to
    # 0-based, to use with Genome
    chrom = np.array(vs["chrom"])
    n = len(chrom)
    pos = np.array(vs["pos"]) - 1
    start = pos - window_size // 2
    end = pos + window_size // 2
    seq_fwd = [genome("chr" + chrom[i], start[i], end[i]) for i in range(n)]
    seq_rev = [str(Seq(x).reverse_complement()) for x in seq_fwd]
    seq_fwd = np.array([list(seq.upper()) for seq in seq_fwd], dtype="object")
    seq_rev = np.array([list(seq.upper()) for seq in seq_rev], dtype="object")
    assert seq_fwd.shape[1] == window_size
    assert seq_rev.shape[1] == window_size
    ref_fwd = np.array(vs["ref"])
    alt_fwd = np.array(vs["alt"])
    ref_rev = np.array([str(Seq(x).reverse_complement()) for x in ref_fwd])
    alt_rev = np.array([str(Seq(x).reverse_complement()) for x in alt_fwd])
    pos_fwd = window_size // 2
    pos_rev = pos_fwd - 1 if window_size % 2 == 0 else pos_fwd

    def prepare_output(seq, pos, ref, alt):
        assert (seq[:, pos] == ref).all(), f"{seq[:, pos]}, {ref}"
        seq_ref = seq
        seq_alt = seq.copy()
        seq_alt[:, pos] = alt
        return (
            tokenize(["".join(x) for x in seq_ref]),
            tokenize(["".join(x) for x in seq_alt]),
        )

    res = {}
    res["input_ids_ref_fwd"], res["input_ids_alt_fwd"] = prepare_output(seq_fwd, pos_fwd, ref_fwd, alt_fwd)
    res["input_ids_ref_rev"], res["input_ids_alt_rev"] = prepare_output(seq_rev, pos_rev, ref_rev, alt_rev)
    return res

In [ ]:
if average_rc:
    dataset.set_transform(get_tokenized_seq_average_rc)
else:
    dataset.set_transform(get_tokenized_seq)

In [ ]:
dataset[0]

## Run inference

In [ ]:
training_args = TrainingArguments(
    output_dir=tempfile.TemporaryDirectory().name,
    per_device_eval_batch_size=128,
    # pyfaidx does not allow multiple workers
    # for longer jobs you can use the gpn.data.Genome class,
    # which loads a local fasta file into memory and can use multiple
    # workers, see e.g.
    # https://github.com/songlab-cal/gpn/blob/main/gpn/ss/run_vep_embed_dist.py
    dataloader_num_workers=0,
    remove_unused_columns=False,
    torch_compile=False,
    fp16=True,
    report_to="none",
)
trainer = Trainer(model=model, args=training_args)
preds = trainer.predict(test_dataset=dataset).predictions
preds.shape

## Compute metrics

In [ ]:
V["score"] = preds

In [ ]:
# let's compare to the precomputed scores
# this is using RC averaging
V["precomputed_score"] = pd.read_parquet("hf://datasets/songlab/TraitGym/mendelian_traits_matched_9/features/GPN_final_EuclideanDistance.parquet").score.values
V[["score", "precomputed_score"]].corr()

In [ ]:
plt.figure(figsize=(2, 2))
sns.histplot(
    data=V, x="score", bins=30, hue="label", stat="density",
    common_norm=False, common_bins=True,
)
sns.despine();

In [ ]:
# global AUPRC
average_precision_score(V.label, V.score)

In [ ]:
# AUPRC by chrom
res_by_chrom = []
for chrom in V.chrom.unique():
    V_chrom = V[V.chrom == chrom]
    res_by_chrom.append([chrom, len(V_chrom), average_precision_score(V_chrom.label, V_chrom.score)])
res_by_chrom = pd.DataFrame(res_by_chrom, columns=["chrom", "n", "AUPRC"])
res_by_chrom

In [ ]:
# Weighted average

def stat(df):
    weight = df["n"] / df["n"].sum()
    return (df["AUPRC"] * weight).sum()

stat(res_by_chrom)

In [ ]:
def bootstrap_se(df, stat, n_bootstraps=1000):
    df = pl.DataFrame(df)
    return (
        pl.Series([
            stat(df.sample(len(df), with_replacement=True, seed=i))
            for i in range(n_bootstraps)]
        )
        .std()
    )

bootstrap_se(res_by_chrom, stat)

In [ ]:
# compare with official results (which use RC averaging)
pd.read_csv("hf://datasets/songlab/TraitGym/mendelian_traits_matched_9/AUPRC_by_chrom_weighted_average/all/GPN_final_EuclideanDistance.plus.score.csv")